In [1]:
#Paper implementation

In [2]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import imageio
from albumentations import VerticalFlip, HorizontalFlip, ElasticTransform, GridDistortion, OpticalDistortion, CoarseDropout, ShiftScaleRotate, Compose, RandomRotate90

In [3]:
# takes a path and creates a directory if it does not exist
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [4]:
def load_data(path):
    """X = images and Y = masks """
    train_x = sorted(glob(os.path.join(path, "train", "*.*")))
    train_y = sorted(glob(os.path.join(path, "train_nonhr", "*.*")))

    test_x = sorted(glob(os.path.join(path, "val", "*.*")))
    test_y = sorted(glob(os.path.join(path, "val_nonhr", "*.*"))) 
    return (train_x, train_y), (test_x, test_y)

In [5]:
def augment_data(images, save_path, augment=True):
    H = 512
    W = 512

    for idx, x in tqdm(enumerate(images), total=len(images)):
        """Extracting names"""
        name = x.split("\\")[-1].split(".")[0]

        """Reading image and mask"""
        x = cv2.imread(x, cv2.IMREAD_COLOR)

        if augment:
            
            aug = RandomRotate90(p=1.0)
            augmented = aug(image=x)
            x4 = augmented['image']
            
#Horizontal Flip
            aug = HorizontalFlip(p=1.0)
            augmented = aug(image=x)
            x1 = augmented["image"]
            
            aug = RandomRotate90(p=1.0)
            augmented = aug(image=x1)
            x7 = augmented['image']
            
#Vertical Flip
            aug = VerticalFlip(p=1.0)
            augmented = aug(image=x)
            x2 = augmented["image"]
            
            aug = RandomRotate90(p=1.0)
            augmented = aug(image=x2)
            x10 = augmented['image']         
            
            X = [x, x1, x2, x4, x7, x10]

        else:
            X = [x]

        index = 0
        for image in X:

            if len(X) == 1:
                tmp_image_name = f"{name}.jpg"
            else:
                tmp_image_name = f"{name}_{index}.jpg"

            image_path = os.path.join(save_path, tmp_image_name)

            """Saving the image"""
            cv2.imwrite(image_path, image)

            index += 1

In [6]:
if __name__ == "__main__":
    """Seeding"""
    np.random.seed(42)

    """Load the data"""
    data_path = "Datasets/HR/HR_preprocessed/"
    (train_x, train_y), (test_x, test_y) = load_data(data_path)
    
    """DATA AUGMENTATION"""
    # Creating directories
    create_dir("Datasets/HR/HR_aug/train")
    create_dir("Datasets/HR/HR_aug/train_nonhr")
    create_dir("Datasets/HR/HR_aug/val")
    create_dir("Datasets/HR/HR_aug/val_nonhr")

    augment_data(train_x, "Datasets/HR/HR_aug/train", augment=True)
    augment_data(train_y, "Datasets/HR/HR_aug/train_nonhr", augment=True)
    augment_data(test_x, "Datasets/HR/HR_aug/val", augment=True)
    augment_data(test_y, "Datasets/HR/HR_aug/val_nonhr", augment=True)

100%|██████████| 81/81 [00:34<00:00,  2.35it/s]
